In [15]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: SkiaSharp"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.6.3-alpha"
#r "nuget: System.Linq.Async"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;


Installed Packages Microsoft.SemanticKernel, 1.15.1 Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.6.3-alpha SkiaSharp, 2.88.8 System.Linq.Async, 6.0.1

Loading extensions from `C:\Users\mikokono\.nuget\packages\skiasharp\2.88.8\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [18]:
var settings  = Settings.LoadFromFile("config/settings.json");
#pragma warning disable SKEXP0001, SKEXP0020, SKEXP0010
var memory = new MemoryBuilder()
                    .WithAzureOpenAITextEmbeddingGeneration("ada-002", settings.azureEndpoint, settings.apiKey)
                    .WithMemoryStore(new AzureAISearchMemoryStore(settings.aisearchendpoint, settings.aisearchkey))
                    .Build();

In [19]:
using System.IO;



var poemParagraphs = File.ReadAllText("data/poem.txt").Split(new string[] { "\r\n\r\n", "\n\n" }, StringSplitOptions.RemoveEmptyEntries);


In [20]:
using System.Collections;
using System.Linq;


foreach (var paragraph in poemParagraphs)
{
    await memory.SaveInformationAsync(
        collection: "PoemParagraphs",
        text: paragraph,
        id: paragraph.GetHashCode().ToString(),
        description: paragraph
    );
}

Error: Microsoft.SemanticKernel.HttpOperationException: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.
Status: 404 (Not Found)
ErrorCode: DeploymentNotFound

Content:
{"error":{"code":"DeploymentNotFound", "message":"The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again."}}

Headers:
apim-request-id: REDACTED
x-ms-client-request-id: 1c45906d-fb1c-4ba6-9cd3-6d4043611a56
Strict-Transport-Security: REDACTED
X-Content-Type-Options: REDACTED
x-ms-region: REDACTED
Date: Thu, 04 Jul 2024 17:27:19 GMT
Content-Length: 198
Content-Type: application/json

 ---> Azure.RequestFailedException: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.
Status: 404 (Not Found)
ErrorCode: DeploymentNotFound

C

In [ ]:
var question = new[]{"How do I get started?", "Can I build a chat with SK?"};
foreach(var q in question)
{
    Console.WriteLine($"Searching for: {q}");
    var results = await memory.SearchAsync("GitHubFiles", q, limit: 1).ToListAsync();
    var result = results.FirstOrDefault();
    if (result != null)
    {
        Console.WriteLine("URL:     : " + result.Metadata.Id);
        Console.WriteLine("Title    : " + result.Metadata.Description);
        Console.WriteLine("Relevance: " + result.Relevance);
    }
    Console.WriteLine();
}